In [ ]:
# @title # ⚡ AutoQuant

# @markdown > 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

# @markdown ❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

# @markdown **Usage:** Download the model by **running this cell** and then run the cells corresponding to your quantization methods of interest.

# @markdown To quantize a 7B model, GGUF only needs a T4 GPU, while the other methods require an A100 GPU.

# @markdown *See also the [AutoQuantize](https://colab.research.google.com/drive/1Li3USnl3yoYctqJLtYux3LAIy4Bnnv3J) notebook from zainulabideen.*

# @markdown ---

# @markdown ## 🤗 Download model (required)
# @markdown `HF_TOKEN` corresponds to the name of the secret that stores your [Hugging Face access token](https://huggingface.co/settings/tokens) in Colab.

MODEL_ID = "mlabonne/Zebrafish-7B" # @param {type:"string"}
USERNAME = "mlabonne" # @param {type:"string"}
HF_TOKEN = "HF_TOKEN" # @param {type:"string"}

MODEL_NAME = MODEL_ID.split('/')[-1]

# Download model
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}
!pip install -q huggingface_hub

from huggingface_hub import create_repo, HfApi, ModelCard
from google.colab import userdata, runtime

# Defined in the secrets tab in Google Colab
hf_token = userdata.get(HF_TOKEN)
api = HfApi()

In [ ]:
# @title ## 🧩 GGUF

# @markdown Quantization methods: `iqs_2`, `iqs_3`, `q2_k`, `q3_k_l`, `q3_k_m`, `q3_k_s`, `q4_0`, `q4_1`, `q4_k_m`, `q4_k_s`, `q5_0`, `q5_1`, `q5_k_m`, `q5_k_s`, `q6_k`, `q8_0`, etc.

# @markdown Learn more about GGUF and quantization methods in [this article](https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html).

QUANTIZATION_FORMAT = "q5_k_m" # @param {type:"string"}
QUANTIZATION_METHODS = QUANTIZATION_FORMAT.replace(" ", "").split(",")

# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!python llama.cpp/convert.py {MODEL_NAME} --outtype f16 --outfile {fp16}

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/quantize {fp16} {qtype} {method}

# Create model card
card = ModelCard.load(MODEL_ID)
card.data.tags.append("autoquant")
card.data.tags.append("gguf")
card.save(f'{MODEL_NAME}/README.md')

# Upload model
create_repo(
    repo_id = f"{USERNAME}/{MODEL_NAME}-GGUF",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)
api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{USERNAME}/{MODEL_NAME}-GGUF",
    allow_patterns=["*.gguf","$.md"],
    token=hf_token
)

In [ ]:
# @title ## 🧠 GPTQ

# @markdown Learn more about the GPTQ algorithm in [this article](https://mlabonne.github.io/blog/posts/4_bit_Quantization_with_GPTQ.html).

!pip install -qqq auto-gptq optimum accelerate

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

BITS = 4 # @param {type:"integer"}
GROUP_SIZE = 128 # @param {type:"integer"}
DAMP_PERCENT = 0.1 # @param {type:"number"}

# Quantize model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
quantization_config = GPTQConfig(bits=BITS, dataset="c4", tokenizer=tokenizer, damp_percent=DAMP_PERCENT)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto", quantization_config=quantization_config, low_cpu_mem_usage=True)

# Save model and tokenizer
save_folder = MODEL_ID + "-GPTQ"
model.save_pretrained(save_folder, use_safetensors=True)
tokenizer.save_pretrained(save_folder)

# Create model card
card = ModelCard.load(MODEL_ID)
card.data.tags.append("autoquant")
card.data.tags.append("gptq")
card.save(f'{save_folder}/README.md')

# Upload model
create_repo(
    repo_id = f"{USERNAME}/{MODEL_NAME}-GPTQ",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)
api.upload_folder(
    folder_path=save_folder,
    repo_id=f"{USERNAME}/{MODEL_NAME}-GPTQ",
    token=hf_token
)

In [ ]:
# @title # 🦙 ExLlamaV2

# @markdown Learn more about ExLlamaV2 in [this article](https://mlabonne.github.io/blog/posts/ExLlamaV2_The_Fastest_Library_to_Run%C2%A0LLMs.html).

BPW = 5.0 # @param {type:"number"}

# Install ExLLamaV2
!git clone https://github.com/turboderp/exllamav2
!pip install -e exllamav2
!cp {MODEL_NAME} base_model
!rm base_mode/*.bin

# Download dataset
!wget https://huggingface.co/datasets/wikitext/resolve/9a9e482b5987f9d25b3a9b2883fc6cc9fd8071b3/wikitext-103-v1/wikitext-test.parquet

# Quantize model
save_folder = MODEL_ID + "-EXL2"
!mkdir {save_folder}
!python exllamav2/convert.py \
    -i base_model \
    -o {save_folder} \
    -c wikitext-test.parquet \
    -b {BPW}

# Copy files
!rm -rf quant/out_tensor
!rsync -av --exclude='*.safetensors' --exclude='.*' ./base_model/ ./{save_folder}/

# Create model card
card = ModelCard.load(MODEL_ID)
card.data.tags.append("autoquant")
card.data.tags.append("exl2")
card.save(f'{save_folder}/README.md')

# Upload model
create_repo(
    repo_id = f"{USERNAME}/{MODEL_NAME}-{BPW:.1f}bpw-exl2",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)
api.upload_folder(
    folder_path=save_folder,
    repo_id=f"{USERNAME}/{MODEL_NAME}-{BPW:.1f}bpw-exl2",
    token=hf_token
)

In [ ]:
# @title ## ⚖️ AWQ

# @markdown See the [AutoAWQ repository](https://github.com/casper-hansen/AutoAWQ) for more information.

# Install AutoAWQ
!pip install -qqq -U https://github.com/casper-hansen/AutoAWQ/releases/download/v0.2.4/autoawq-0.2.4+cu118-cp310-cp310-linux_x86_64.whl
!pip install zstandard

from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

BITS = 4 # @param {type: "integer"}
GROUP_SIZE = 128 # @param {type: "integer"}
VERSION = "GEMM" # @param {type: "string"}
ZERO_POINT = True # @param {type: "boolean"}

quant_config = {
    "w_bit": BITS,
    "q_group_size": GROUP_SIZE,
    "version": VERSION,
    "zero_point": ZERO_POINT
}
save_folder = MODEL_NAME + "-AWQ"

# Quantize model
model = AutoAWQForCausalLM.from_pretrained(MODEL_NAME, safetensors=True, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model.quantize(tokenizer, quant_config=quant_config)

# Save model and tokenizer
model.save_quantized(save_folder)
tokenizer.save_pretrained(save_folder)

# Create model card
card = ModelCard.load(MODEL_ID)
card.data.tags.append("autoquant")
card.data.tags.append("awq")
card.save(f'{save_folder}/README.md')

# Upload model
create_repo(
    repo_id = f"{USERNAME}/{MODEL_NAME}-AWQ",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)
api.upload_folder(
    folder_path=save_folder,
    repo_id=f"{USERNAME}/{MODEL_NAME}-AWQ",
    token=hf_token
)

In [ ]:
# @title ## 🐘 HQQ

# @markdown See the official [HQQ repository](https://github.com/mobiusml/hqq) for more information.

!git clone https://github.com/mobiusml/hqq.git
!pip install -e hqq
!python hqq/kernels/setup_cuda.py install
!pip install flash-attn --no-build-isolation
!pip install transformers --upgrade
!num_threads=8; OMP_NUM_THREADS=$num_threads CUDA_VISIBLE_DEVICES=0

import torch
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *

BITS = 2 # @param {type:"integer"}
GROUP_SIZE = 128 # @param {type:"integer"}

# Quant config
quant_config = BaseQuantizeConfig(
    nbits=BITS,
    group_size=GROUP_SIZE
)

# Quantize model
model = HQQModelForCausalLM.from_pretrained(
    MODEL_ID,
    cache_dir=".",
    attn_implementation="flash_attention_2"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model.quantize_model(quant_config=quant_config, device='cuda')

# Save model and tokenizer
save_folder = MODEL_ID + "-HQQ"
model.save_quantized(save_folder)
tokenizer.save_pretrained(save_folder)

# Create model card
card = ModelCard.load(MODEL_ID)
card.data.tags.append("autoquant")
card.data.tags.append("hqq")
card.save(f'{save_folder}/README.md')

# Upload model
create_repo(
    repo_id = f"{USERNAME}/{MODEL_NAME}-{BITS}bit-HQQ",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)
api.upload_folder(
    folder_path=save_folder,
    repo_id=f"{USERNAME}/{MODEL_NAME}-{BITS}bit-HQQ",
    token=hf_token
)